# Train all models part 1 
- ds08
- fc 1,2,3
- h0, h1
- 6 total models
- use set 3 (2, w, 0)

In [11]:
import os
import pickle
import h5py
import time
import matplotlib
import numpy as np
print(np.__version__)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy.stats as stats
import math
import random
import gc
import multiprocessing as mp
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

# local files
import make_sets
import models
import utils
import training

1.19.5


### load all data and labels (not v)

In [5]:
y_labels = []
with open("data_csv/y_labels.txt", "r") as f:
    for l in f:
        y_labels.append(l.strip())
        
t_labels = []
with open("data_csv/t_labels.txt", "r") as f:
    for l in f:
        t_labels.append(l.strip())

v_labels = []
with open("data_csv/v_labels.txt", "r") as f:
    for l in f:
        v_labels.append(l.strip())
        
df = pd.read_csv('data_csv/df08_all_resampled_interp_h0.csv')
df.drop(columns=[df.columns[0]], inplace=True)
_t_labels = t_labels[:]
_t_labels.remove('P2')
_t_labels.remove('P50')
df.drop(columns=_t_labels, inplace=True)
df.drop(columns=v_labels, inplace=True)
df.head()

,unit,cycle,Fc,hs,ui,dataset,y,P2,P50,fan_eff_mod,fan_flow_mod,LPC_eff_mod,LPC_flow_mod,HPC_eff_mod,HPC_flow_mod,HPT_eff_mod,HPT_flow_mod,LPT_eff_mod,LPT_flow_mod
0,1.0,31.0,1.0,0.0,0,DS08a-009,41.000000,13.796511,16.155448,-0.001481,-0.000917,-0.001461,-0.00157,-0.000486,-0.001469,-0.000947,-0.000781,-0.001757,-0.001559
1,1.0,31.0,1.0,0.0,0,DS08a-009,40.999070,13.786445,16.130395,-0.001481,-0.000917,-0.001461,-0.00157,-0.000486,-0.001469,-0.000947,-0.000781,-0.001757,-0.001559
2,1.0,31.0,1.0,0.0,0,DS08a-009,40.998140,13.774472,16.105795,-0.001481,-0.000917,-0.001461,-0.00157,-0.000486,-0.001469,-0.000947,-0.000781,-0.001757,-0.001559
3,1.0,31.0,1.0,0.0,0,DS08a-009,40.997209,13.768953,16.085989,-0.001481,-0.000917,-0.001461,-0.00157,-0.000486,-0.001469,-0.000947,-0.000781,-0.001757,-0.001559
4,1.0,31.0,1.0,0.0,0,DS08a-009,40.996279,13.762214,16.066372,-0.001481,-0.000917,-0.001461,-0.00157,-0.000486,-0.001469,-0.000947,-0.000781,-0.001757,-0.001559


In [9]:
dataset_dirs = [
    'experiments/lstm_rul_ds08/',
    'experiments/lstm_rul_ds01-07/'
]
di = 0

health_states = [
    'h0/',
    'h1/'
]
hi = 1

flight_classes = [
    'fc1/',
    'fc2/',
    'fc3/'
]
fi = 0

for hi in range(0, len(health_states)):
    for fi in range(0, len(flight_classes)):
        make_sets.prepare_for_training(df=df[df.Fc==fi+1],
                                       lookback=10,
                                       horizon=1,
                                       n_out=1,
                                       save_files=True,
                                       directory=dataset_dirs[di]+health_states[hi]+flight_classes[fi])

unit unique identifier (ui): [0, 2, 8, 9, 14]
number of units: 5
train, val, test set counts: 3, 1, 1
train ui: [8, 9, 14]
val ui: [2]
test ui: [0]
unit unique identifier (ui): [3, 4, 6, 7, 10, 20, 21, 22, 23, 24]
number of units: 10
train, val, test set counts: 6, 2, 2
train ui: [4, 23, 24, 6, 20, 10]
val ui: [3, 21]
test ui: [7, 22]
unit unique identifier (ui): [1, 5, 11, 12, 13, 15, 16, 17, 18, 19]
number of units: 10
train, val, test set counts: 6, 2, 2
train ui: [13, 19, 18, 16, 11, 1]
val ui: [17, 15]
test ui: [5, 12]


In [ ]:
lookback = 10

di = 0
hi = 0
for fi in range(0, len(flight_classes)):
    current_dir = directory=dataset_dirs[di]+health_states[hi]+flight_classes[fi]
    print(f"[INFO] current directory: <{current_dir}>")
    
    X_train = np.load(current_dir + 'X_train.npy')
    X_val = np.load(current_dir + 'X_val.npy')
    X_test = np.load(current_dir + 'X_test.npy')

    y_train = np.load(current_dir + 'y_train.npy')
    y_val = np.load(current_dir + 'y_val.npy')
    y_test = np.load(current_dir + 'y_test.npy')

    X_train = np.concatenate([X_train, X_val, X_test], axis=0)
    y_train = np.concatenate([y_train, y_val, y_test], axis=0)
    
    print("[INFO] getting LSTM model")
    model = models.lstm(lookback=lookback,
                       n_features=X_train.shape[2],
                       n_out=y_train.shape[2])
    
    print(f"[INFO] training <{current_dir}>")
    training.train(X_train=X_train,
                   y_train=y_train,
                   directory=current_dir,
                   input_name='',
                   model=model,
                   early_stop=True,
                   min_delta=.1,
                   patience=50,
                   monitor='loss',
                   batch_size=64,
                   epochs=1000,
                   lr=.0001)

[INFO] current directory: <experiments/lstm_rul_ds08/h0/fc1/>
[INFO] getting LSTM model
